In [ ]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser 
import bs4
import os  # 환경 변수 설정을 위한 모듈


# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = ""


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:

# 1. 뉴스기사 내용을 웹에서 로드하는 단계
# - WebBaseLoader를 사용하여 특정 웹사이트(네이버 뉴스 기사)에서 텍스트를 가져온다.
# - bs4.SoupStrainer를 사용하여 필요한 부분(div 태그의 특정 클래스)을 선택적으로 파싱한다.
# 조건: 크롤링할 네이버 뉴스 기사를 다른 기사로 바꿔보세요.
# 참고: 에러가 나는 경우 뉴스 기사에서 F12(개발자 도구)를 눌러 기사 제목과 내용이 담긴 올바른 클래스명을 가져오세요.
# 참고: 저작권으로 인해 기사 자체에서 크롤링이 막혀 있을 수도 있습니다. 
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/296/0000087319?sid=103",),  # 크롤링할 네이버 뉴스 링크 입력
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

# 뉴스 기사 내용을 로드하여 문서 형태로 저장
docs = loader.load()
print(f"문서의 수: {len(docs)}")  # 로드된 문서 개수 확인

문서의 수: 1


In [3]:

# 2. 불러온 뉴스 기사를 청크(chunk)로 나누고, 인덱싱
# - RecursiveCharacterTextSplitter를 사용하여 긴 텍스트를 일정 크기의 청크로 나눈다.
# - chunk_size: 한 개의 청크 크기 (문자 수)
# - chunk_overlap: 청크 간 겹치는 부분의 크기 (연속성 유지)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 여러 개의 작은 청크로 나눔
splits = text_splitter.split_documents(docs)
print(f"청크 개수: {len(splits)}")  # 생성된 청크 개수 확인


청크 개수: 4


In [4]:

# 3. 벡터스토어(Vector Store)를 생성
# - FAISS(Facebook AI Similarity Search)를 사용하여 벡터 DB를 생성
# - OpenAIEmbeddings를 사용하여 텍스트를 벡터로 변환
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_47124\1861515578.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))


In [5]:

# 4. 뉴스 기사에서 정보를 검색할 수 있도록 검색기(retriever) 생성
retriever = vectorstore.as_retriever()


In [6]:

# 5. 프롬프트 템플릿 정의
# - 검색된 문맥(Context)과 사용자 질문(Question)을 조합하여 LLM에 입력
# - 문맥에서 답을 찾을 수 없으면 적절한 메시지를 출력
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)에서 주어진 질문(question)에 답하는 것입니다.
검색된 다음 문맥(context)을 사용하여 질문(question)에 답하세요. 만약, 주어진 문맥(context)에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다`라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)



In [10]:
# 6. LLM 모델 선택
# - OpenAI의 GPT-4o-mini 모델을 사용하여 검색된 정보를 바탕으로 응답을 생성
# - temperature=0 설정으로 일관된 답변을 생성하도록 설정
llm = ChatOpenAI(model_name="gpt-5-nano", temperature=1)



In [11]:
# 7. RAG(Retrieval-Augmented Generation) 체인 생성
# - 사용자의 질문을 받아 검색(retriever)을 수행한 후, 결과를 프롬프트(prompt)와 결합
# - 결합된 입력을 LLM에 전달하고, 응답을 출력 형식으로 변환
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)



In [12]:
# 실행 예시 (사용자가 입력한 질문)
user_question = "이 뉴스에서 다룬 주요 이슈는 무엇인가요?"
response = rag_chain.invoke(user_question)
print(response)  # 최종 생성된 답변 출력

주요 이슈는 씹는 재질이 뇌 화학 및 인지 기능에 미치는 영향에 관한 연구 결과입니다. 구체적으로는 딱딱한 물체(나무 막대기)를 씹는 것이 전대상피질에서 글루타치온 같은 항산화제 수치를 증가시키고 기억력의 일부 과제 성과와 양의 연관을 보였으며, 껌을 씹는 그룹은 유의미한 변화가 없었다는 점이 핵심 내용입니다.


In [13]:
# 실행 예시 (사용자가 입력한 질문)
user_question = "뉴스 내용 담겨 있는 질문 넣어보기"
response = rag_chain.invoke(user_question)
print(response)  # 최종 생성된 답변 출력

주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다


In [14]:
# 실행 예시 (사용자가 입력한 질문)
user_question = "뉴스 내용을 요약하여 영어로 번역해주세요."
response = rag_chain.invoke(user_question)
print(response)  # 최종 생성된 답변 출력

다음은 주어진 뉴스 내용을 영어로 요약한 번역문입니다:

A Kyungpook National University–led study reported through PsyPost examined whether chewing harder materials, rather than gum, affects brain chemistry and memory. Involving 52 healthy college students who were split into two groups, one group chewed standard paraffin wax gum while the other chewed small wooden sticks similar to tongue depressors, for five minutes. During chewing, participants followed a pace of one chew per second, alternating 30 seconds of chewing with 30 seconds of rest. Brain glutathione levels in the prefrontal cortex were measured before and after chewing using magnetic resonance spectroscopy, and participants also completed cognitive tests assessing memory, attention, language, and visuospatial abilities.

Results showed that the wooden-stick group experienced a significant increase in glutathione after chewing, suggesting higher antioxidant levels in a brain region important for cognitive control. By contrast, the gum group did no